In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
import tabulate
from IPython.display import HTML, display, Markdown
import csv
import unicodedata
import getopt
from ipywidgets import HBox, VBox, Layout
from matplotlib.ticker import MaxNLocator

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args


def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_9R_8D_NZ.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))
    

# Industrial Sector Overview <a class="anchor" id="buildings"></a>
The industrial sector in the OEO database is subdivided into the manufacturing and non-manufacturing sectors. Their representation in the OEO database consists of two key components: 1) end-use service demands and 2) the demand technologies/processes used to meet those demands.

## Service Demands <a class="anchor" id="service_demands"></a>

End-use demands in the industrial sector of the OEO database are aggregated based on the [North American Industry Classification System (NAICS)](https://www.census.gov/naics/). Table 1 shows the end-use demands represented, mapped to their NAICS codes, along with their short codes in the OEO database. We developed representations of the manufacturing sector as well as mining, construction and agriculture in the non-manufacturing sector as approximately 99% of industrial energy consumption was accounted for by these sectors as [per the EIA](https://www.eia.gov/energyexplained/use-of-energy/industry.php). The demands for these industrial sub-sectors are represented in estimates of annual billion dollars of production shipment value with the exception of cement manufacturing, where demands are instead represented in million metric tons of cement production. These demands are derived from the [Manufacturing Energy Consumption Survey (MECS)](https://www.eia.gov/consumption/manufacturing/) for the base year of 2014 on a U.S. state-by-state basis. The demands are then aggregated by the OEO regions as shown in the overview jupyter notebook. 

Sub-sector specific growth rates are applied to the end-use demands in the industrial sector. For example, growth in demand in the petroleum refining and the iron and steel mills and ferroalloys sub-sectors are assumed to be flat based on the refinery utilization and iron and steel manufacturing projections and in the AEO2022 reference case. The demand in plastic and rubber products is assumed to increased 3% annually from 2020 to 2050 based on [Meys et al. (2021)](https://www.science.org/doi/10.1126/science.abg9853). The growth in demand of cement manufacturing is also taken from AEO 2022 reference case. Finally, the growth rates of all other manufacturing and non-manufacturing sub-sectors, not explicitly specified, are assumed to increase at an annual rate of 1%.

**Table 1.** Annual service demand categories as represented in the OEO database and their corresponding North American Industry Classification System (NAICS) codes.

| Sub-sector | End-Use Demand | NAICS Code | OEO Demand Commodity |
|:-|:-|:-|:-|
| Manufacturing | Food and Beverage <img width=15/> | 311+312 |INDFOOD<img width=15/>|
| Manufacturing | Pulp, Paper, & Paperboard Mills | 322 | INDPPP |
| Manufacturing | Petroleum Refining | 324 | INDPR |
| Manufacturing | Chemical Manufacturing | 325 | INDCM |
| Manufacturing | Plastics and Rubber Products | 326 | INDPRP |
| Manufacturing | Cement Manufacturing | 327310 | INDCEM |
| Manufacturing | Iron and Steel Mills and Ferroalloys | 331110 | INDISF |
| Manufacturing | Alumina and Aluminum | 3313 | INDAA |
| Manufacturing | All other manufacturing | 313-316, 321, 323, 327, 331-337, 339 | INDOTH |
| Non-Manufacturing | Agriculture – Crops | 111 | INDNMAGCROPS |
| Non-Manufacturing | Agriculture – Other | 112-5 | INDNMAGOTH |
| Non-Manufacturing | Coal Mining | 2121 | INDNMCOALMIN |
| Non-Manufacturing | Oil and gas mining | 211 | INDNMNGMIN |
| Non-Manufacturing | Metal and other non metal mining | 2122-2123| INDNMMETMIN |
| Non-Manufacturing | Construction | 23| INDNMCONST |

## Demand Technology Specification <a class="anchor" id="demand_technology"></a>

Unlike the other major sectors in the OEO database, explicit end-use technology options with costs and efficiencies are not depicted for most of the industrial sub-sectors. Instead, we consider the major energy consumed by a set of common industrial processes in order to account for the heterogeneity across the industrial sector. These industrial processes include: 1) process heating, 2) conventional boiler use, 3) combined heat and power or co-generation systems, 4) machine drives, 5) facility heating ventilation and air conditioning systems, 6) process cooling and refrigeration and 7) a catch-all 'other' energy use category. 

### Efficiency <a class="anchor" id="efficiency"></a>

As in the other major sectors, the efficiency parameter has two functions in the OEO database: (1) specify the conversion efficiency of each process, and (2) specify linkages in input and output processes to build the energy system network. The industrial processes are modeled in two stages: The first stage contains a representation of the seven industrial processes listed above along with their average efficiencies with respect to a specific end-use industrial sub-sector. For example, the efficiency of combined heat and power processes in the alumina and aluminum manufacturing industry is distinct from those in the food manufacturing industry. In the second stage, we represent technology efficiencies defined as the output end-use demands (which are typically in billion dollars) normalized by the output energy from the industrial processes identified above. The [EIA manufacturing energy consumption survey (MECS)](https://www.eia.gov/consumption/manufacturing/) provides the framework for fuel use by industrial process. In this way, we only fix the amount of secondary energy required to meet any given end-use demand but the technologies used to generate the secondary energy can be optimized endogenously to the model. Future work will focus on explicit technology characterization, with emphasis on technology improvements and other innovative technologies to support further decarbonization in the industrial sector.

In [2]:
def show_efficiency(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df_tech_desc[['tech_desc']] = df_tech_desc[['tech_desc']].fillna('to be updated')
    df = pd.read_sql("SELECT regions, input_comm, tech, vintage, efficiency FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector=='industrial')", con)
    
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df#df.drop("vintage", axis=1).groupby(by = ['regions','agg_tech']).sum().reset_index()
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
#     techs = ['All'] + list(df_sum.agg_tech.unique())
    
    df_sum = df_sum.loc[~df_sum['tech'].str.contains('BLND')].copy() # remove blending tech efficiencies
    df_sum = df_sum.loc[~df_sum['tech'].str.contains('_emissions')].copy() # remove blending tech efficiencies
    df_sum = df_sum.loc[~df_sum['agg_tech'].str.contains('accounting')].copy() # remove blending tech efficiencies
    
    techs = df_sum.agg_tech.unique()
    regions = df_sum.regions.unique()

    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
            df_sel.efficiency = 100*df_sel.efficiency
        df_sel = df_sel[['regions', 'input_comm', 'tech','agg_tech','vintage','efficiency']].drop_duplicates()
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "input", "technology", "description","vintage",'efficiency (%)' ], floatfmt=".1f", tablefmt='html')))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)

    
show_efficiency(con)

### Investment and fixed costs <a class="anchor" id="costinvest"></a>

While costs are generally not specified in our treatment of the industrial sector representation, we consider the cost premiums for new conventional electric boilers (368 \\$2016/kW), electric process heating equipment (368 \\$2016/kW), hydrogen boilers (227 \\$2016/kW), hydrogen process heating equipment (227 \\$2016/kW) across the industrial sub-sectors. These costs are taken from the [Net Zero America report (Annex A3)](https://netzeroamerica.princeton.edu/the-report). These costs help better realize the transition to decarbonization options in traditionally difficult to decarbonize industrial processes in the industrial sector. Fixed cost premiums are also set as 0.6 \\$2018/kW based on the assumption in Annex A3 of the Net Zero America report for the entire lifetime of these boilers and process heating equipment (20 years). In our modeling approach, heavy industries including cement, chemicals, iron and steel mills and ferro-alloys, petroleum refining, and plastics are precluded from use of hydrogen or electricity for process heat. However, we assume that all sectors can electrify conventional boilers.

In [3]:
def show_cost_invest(con):
    
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)

    df = pd.read_sql(
        "SELECT regions, tech,vintage, cost_invest, cost_invest_units FROM CostInvest WHERE tech IN (SELECT tech FROM technologies WHERE sector=='industrial') ORDER BY tech, vintage",
        con)

    display_types = ['table', 'figure']
    
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()


    def filter_tech(tech='', region='', o_format=''):
        if tech == 'All':
            df_sel = df[(df.regions == region)]
        else:
            df_sel = df[(df.agg_tech.str.contains(tech)) & (df.regions == region)]
        if o_format == 'table':
            df_sel = df_sel.pivot_table(
                index=['regions', 'tech', 'agg_tech', 'cost_invest_units'],
                columns='vintage',
                values='cost_invest').reset_index().set_index('regions')
            df_sel.rename(columns={'cost_invest_units': 'units', 'agg_tech':'description'}, inplace=True)
            df_sel['units'] = df_sel['units'].str.replace('#','').str.replace('M$','$M').str.strip()
            if len(df_sel['units'].unique())==1:
                df_sel['units'] = df_sel['units'].unique()[0].replace('M$','$M')
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".0f",
                                      tablefmt='html')))
        elif (o_format == 'figure') & (tech!='All') :
            fig, ax = plt.subplots(figsize=(10, 6))
            for ind_tech in df_sel.tech.unique():
                plt.plot(df_sel[df_sel.tech == ind_tech].vintage,
                         df_sel[df_sel.tech == ind_tech].cost_invest,
                         label=ind_tech)
            plt.legend()
            plt.ylabel('Investment costs (' + df_sel['cost_invest_units'].unique()[0].replace('#','').strip().replace('M$','$M') + ')')
            #plt.ylim([0, df.cost_invest.max() * 1.1])
            plt.xlabel('Vintage')


    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)
    
show_cost_invest(con)

### Technology shares <a class="anchor" id="tech_split"></a>

Two sets of technology shares are imposed on the industrial sector representation - 1) process based shares and 2) fuel based shares. 

The process based shares are based on the EIA MECS data referenced above to ensure that the end-use demands in the industrial sector are being serviced by a combination of the seven industrial processes identified above as per the specifications of the industrial sub-sector in consideration. These shares are defined by the proportion of secondary energy consumption by the industrial processes being fed to the industrial sub-demands. For example, to meet one unit of aluminum and alumina demand in 2050 in the California region, around 30% of the energy demand must come from machine drives, 24% from process heating and 20% from other industrial processes. These splits must be honored in meeting the alumina and aluminum demand on a temporal and spatial scale based on the MECS energy use data. 

Additionally, fuel based shares are imposed on the early model years based on historical data, which are then are progressively relaxed through the model years. A minimum levels of electrification of that observed in the base year (2020) is maintained through the model time periods. Thus, the model has to use a minimum of 2020 electrification levels but can optimize the remaining share of energy consumed. For the other fuels, LPG, diesel fuel oil, residual fuel oil, and biomass based renewable fuels, 80% of their 2020 usage is set as their share in 2025, 60% of their 2020 usage is set as their share in 2030, 40% of their 2020 usage is set as their share in 2035, 20% of their 2020 usage is set as their share in 2040 and fuel shares are completely removed for the years 2045 and 2050. In the case of natural gas, coal, and other fuel, limits of 80 and 60% of 2020 levels are placed in model years 2025 and 2030 respectively and are subsequently full relaxed by 2035.

#### Fuel based technology shares

In [4]:
def show_tech_shares_fuel(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df_tech_desc[['tech_desc']] = df_tech_desc[['tech_desc']].fillna('to be updated')
    
    df = pd.read_sql("SELECT regions, periods, input_comm, tech, ti_split, ti_split_notes FROM TechInputSplit WHERE tech IN (SELECT tech FROM technologies WHERE sector=='industrial')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    
#     df.sort_values(by='ti_split', ascending=False, inplace=True)
    df_sum = df.copy()
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    
    df_sum = df_sum.loc[~df_sum['tech'].str.endswith('BLND')].copy()
    
    techs = list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    all_periods = df_sum.periods.unique()

    def filter_tech(tech ='', region = '', period= ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region) & (df_sum.periods==period)]
    
        df_sel = df_sel[['regions','periods', 'tech','agg_tech', 'input_comm', 'ti_split']].copy()
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "periods", "technology", "description", "input commodity", "technology share" ], floatfmt=".1f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=all_periods)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, period=w3)    
    
    controls_rows(w)
    
show_tech_shares_fuel(con)


#### Technology/Process based technology shares

In [5]:
def show_tech_shares_process(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df_tech_desc[['tech_desc']] = df_tech_desc[['tech_desc']].fillna('to be updated')
    
    df = pd.read_sql("SELECT regions, periods, input_comm, tech, ti_split, ti_split_notes FROM TechInputSplit WHERE tech IN (SELECT tech FROM technologies WHERE sector=='industrial')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    
#     df.sort_values(by='ti_split', ascending=False, inplace=True)
    df_sum = df.copy()
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    
    df_sum = df_sum.loc[df_sum['tech'].str.endswith('BLND')].copy()
    
    techs = list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    all_periods = df_sum.periods.unique()

    def filter_tech(tech ='', region = '', period= ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region) & (df_sum.periods==period)]
    
        df_sel = df_sel[['regions','periods', 'tech','agg_tech', 'input_comm', 'ti_split']].copy()
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "periods", "technology", "description", "input commodity", "technology share" ], floatfmt=".1f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=all_periods)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, period=w3)

    controls_rows(w)
    
show_tech_shares_process(con)


### Emissions

Process carbon dioxide emissions are modeled for chemicals, cement, iron and steel mills and ferroalloy, and alumina and aluminum manufacturing. These are represented in kilo-tons of carbon dioxide emissions over billion dollars of shipment value except in the case of cement production where instead process emissions are in kilo-tons of carbon dioxide emissions over million metric tons of cement production. We conservatively assume the process emission factors remain constant from the 2020 model time period out to 2050. Process based improvements in the future will likely result in decreased emissions in time periods closer to mid-century. However, we currently have not modeled carbon capture technologies that can offset industrial process emissions. This is an area of ongoing work and will be refined in the next version of the OEO report release.

In [6]:
def show_emission_factors(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df_tech_desc[['tech_desc']] = df_tech_desc[['tech_desc']].fillna('to be updated')
    
    df = pd.read_sql("SELECT regions, emis_comm, vintage, input_comm, tech, emis_act, emis_act_units FROM EmissionActivity WHERE tech IN (SELECT tech FROM technologies WHERE sector=='industrial')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    
    df_sum = df.copy()
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    
    df_sum = df_sum.loc[df_sum['tech'].str.endswith('BLND')].copy()
    
    techs = list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()


    def filter_tech(tech =''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech)]
    
        df_sel = df_sel[['regions', 'tech','agg_tech', 'emis_act', 'emis_act_units']].drop_duplicates()
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description", "emission activity", "emission activity units" ], floatfmt=".1f", tablefmt='html')))
    
    w1 = widgets.Select(options=techs)
    
    w = widgets.interactive(filter_tech, tech=w1)

    controls_rows(w)

    
show_emission_factors(con)


## Look-up Tools <a class="anchor" id="look_up_tools"></a>

### Technology/Commodity Description Look-up Tool <a class="anchor" id="description_look_up"></a>
Use the tool below to search for any key words that may describe a technology or commodity of interest (e.g. heating, cooling). The tool provides a list of all the technologies in the database that may be relevant to the query.

In [7]:
w = widgets.Text(value='Food and Beverage')
display(w)
def f(w):
    if len(w)>0:
        df1 = pd.read_sql("SELECT * FROM commodities WHERE comm_desc LIKE '%" + w + "%'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech_desc LIKE '%" + w + "%'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'


        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])
        
        if len(df)>0:
            display(HTML(tabulate.tabulate(df.set_index('comm_tech'),['technology/commodity','type','description'],tablefmt='html')))
        else:
            print('')
    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='Food and Beverage')

Output()

### Network Diagram Look-up Tool  <a class="anchor" id="network_look_up"></a>
Use the [description lookup tool](#description_look_up) above to identify specific commodity or technology names. Type the name in the box below to generate a corresponding network diagram for that commodity or technology. The slider can be used to view different upstream levels of the network diagram.

In [8]:
w = widgets.Text(value='INDFOOD')
display(w)
def f(w):
    if len(w)>0:
        
        df1 = pd.read_sql("SELECT comm_name, comm_desc FROM commodities WHERE comm_name='" + w + "'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'

        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])

    
        if len(df)>0:
            def show_desc(level):
                display(Markdown(df['desc'][0]))
                final_dem = df['comm_tech'][0]
                df_graph = return_flowd_table(final_dem,level)
                args = create_args_flowd(df_graph)
                colors, quick_run_dot_fmt = return_format_colors()
                args.update(colors)
                #o_str = 'rankdir = "LR" ;'
                #r_str = 'rankdir = "LR" ; \n\t size="12,12";'
                #quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
                dot_graph = quick_run_dot_fmt % args
                display(graphviz.Source(dot_graph))
            w2 = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Level:',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
            w = widgets.interactive(show_desc, level=w2)
            controls_rows(w)
        else:
            print('')

    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='INDFOOD')

Output()

###  Technology/Commodity Look-up Tool <a class="anchor" id="tech_lookup"></a>
Use the tool below to retrieve the description for any technology or commodity within the database. Type the commodity or technology name in the box below to view the description. Note that names are case sensitive.

In [9]:
w = widgets.Text(value='INDFOOD')
display(w)
def f(w):
    df = pd.read_sql("SELECT * FROM commodities WHERE comm_name='" + w + "'", con)
    if len(df)==0:
        df = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)

    if len(df)>0:
        try:
            display(Markdown((df['comm_desc'].values[0].replace('#', '').strip())))
        except:
            display(Markdown(df['tech_desc'].values[0].replace('#', '').strip()))
    else:
        print('')


out = widgets.interactive_output(f, {'w': w})
display(out)


Text(value='INDFOOD')

Output()